In [6]:
all_events = spark.read.parquet('/tmp/all_events') # read the entire content of the hdfs parquet file all-events
all_events.show() # show all events

+------+-------------+-----------------+---------------+----------+-----+---------+--------------------+
|Accept|Cache-Control|             Host|     User-Agent|event_type| item|item_type|           timestamp|
+------+-------------+-----------------+---------------+----------+-----+---------+--------------------+
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3|   default| null|     null|2021-04-12 02:24:...|
|   */*|     no-cache|user1.comcast.com|ApacheBench/2.3

In [13]:
# show unique hosts who created events
all_events.registerTempTable('all_events')
all_events_by_example2 = spark.sql("select distinct(Host) dist_host from all_events")
all_events_by_example2.show() # show the distinct hosts

+-----------------+
|        dist_host|
+-----------------+
|    user2.att.com|
|user3.verizon.com|
|user1.comcast.com|
+-----------------+



In [14]:
# show the activities of the user3 from verizon
all_events_by_example3 = spark.sql("select * from all_events where Host='user3.verizon.com' ")
all_events_by_example3.show() # show the query
all_events_by_example3.printSchema() # show the schema

+------+-------------+-----------------+---------------+----------+-----+---------+--------------------+
|Accept|Cache-Control|             Host|     User-Agent|event_type| item|item_type|           timestamp|
+------+-------------+-----------------+---------------+----------+-----+---------+--------------------+
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3|  purchase|sword|     wood|2021-04-12 02:25:...|
|   */*|     no-cache|user3.verizon.com|ApacheBench/2.3

In [9]:
import pandas as pd 
df = all_events.toPandas() # convert to a Panda DataFrame
df.describe() # DataFrame statistics

,Accept,Cache-Control,Host,User-Agent,event_type,item,item_type,timestamp
count,78,78,78,78,78,63,63,78
unique,1,1,3,1,5,2,2,78
top,*/*,no-cache,user1.comcast.com,ApacheBench/2.3,purchase,sword,wood,2021-04-12 02:25:10.468
freq,78,78,38,78,39,44,43,1


In [12]:
df['frequency'] = df['Host'] # add a frequency column to the database
# group by host and event_type, item, item_type and count the number of events
df.groupby(['Host','event_type','item','item_type'], as_index=False)['frequency'].count() 

,Host,event_type,item,item_type,frequency
0,user1.comcast.com,ditch,sword,wood,4
1,user1.comcast.com,purchase,sword,wood,15
2,user1.comcast.com,upgrade,sword,metal,4
3,user2.att.com,purchase,shield,wood,12
4,user2.att.com,sell,shield,metal,3
5,user2.att.com,upgrade,shield,metal,4
6,user3.verizon.com,purchase,sword,wood,12
7,user3.verizon.com,sell,sword,metal,3
8,user3.verizon.com,upgrade,sword,metal,6
